In [ ]:
#file=r'/content/00000355_s003_t000.edf'
from google.colab import drive

drive.mount("/content/gdrive")

In [ ]:
file


In [ ]:
pip install -U mne

In [ ]:
import mne
import scipy
import matplotlib.pyplot as plt

In [ ]:
def nb_channels_file(filename):
    file = filename
    data = mne.io.read_raw_edf(file)
    channels = data.ch_names
    return len(channels)

In [ ]:
nb_channels_file(file)

In [ ]:
#une fonction qui retourne un dictionnaire tel que chaque cle représente un fichier edf 
#et chaque valeur représente le nombre de channels de ce fichier
#pour chaque patient passé avec son ID en paramètre
def patient_files_channels(ID):
    Dict = {}
    keys=fichiers_patient(ID)
    Values=[]
    for file in keys:
       Values.append(nb_channels_file(file))
    Dict = {}
    for i in range(len(keys)):
       Dict[keys[i]] = Values[i]
    return Dict;

In [ ]:
def affiche_signaux_file(file):
    n=nb_channels_file(file)
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
        
    for i in range(n):
        sig=data.get_data()[i]
        plt.plot(sig[:4999])
        plt.title("signal channel",i)
        plt.show()
        
  

In [ ]:
affiche_signaux_file(file)

In [ ]:
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()

In [ ]:
data

In [ ]:
info = data.info
channels = data.ch_names

In [ ]:
raw_data

In [ ]:
raw_data.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(raw_data[33,:90000])
plt.title("Raw EEG, electrode 0, samples 0-4999")
plt.show()

In [ ]:
##########################################
#                                        #
#            FILTRING                    #
#                                        #
##########################################

In [ ]:
#Butterworth filter (Low pass filter)

In [ ]:
def filtrage_signaux_patient_channel(ID,i):
    list_fichiers=fichiers_patient(ID)
    list_fichiers_filtred=[]
    for fichier in list_fichiers:
            data = mne.io.read_raw_edf(fichier)
            raw_data = data.get_data()
            sig=data.get_data()[i]
            
            cutoff=40.
            fs=1000
            nyq = 0.5 * fs
            low = cutoff / nyq
            b, a = scipy.signal.butter(3, low, btype='low', analog=False)
            filtered_signal=scipy.signal.filtfilt(b,a,sig,axis=0)
            list_fichiers_filtred.append(filtered_signal)
        
    return list_fichiers_filtred  #retourne une liste d'array 

In [ ]:
def filtrage_file_channel(file,i):  #i:channel
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    cutoff=40.
    fs=1000
    nyq = 0.5 * fs
    low = cutoff / nyq
    b, a = scipy.signal.butter(3, low, btype='low', analog=False)
    sig=data.get_data()[i]
    filtered_signal1=scipy.signal.filtfilt(b,a,sig,axis=0)
    return filtered_signal1   #retourne un array (un signal filtré)

In [ ]:
filt=filtrage_file_channel(file,0)

In [ ]:
plt.plot(raw_data[0][:90000])
plt.title("original signal")
plt.show()
plt.plot(filt[:90000])
plt.title("filtered signal : Lowpass Butterworth filter ")
plt.show()

In [ ]:
#FFT filter (smoothing filter)

In [ ]:
import numpy as np
import scipy.fftpack
def filtrage_file_channel_FFT(i):  #i:channel
   # data = mne.io.read_raw_edf(file)
   # raw_data = data.get_data()
    sig=i
    
    sig_fft = scipy.fftpack.fft(sig)
    time_step = 0.02
    period = 5.
    power = np.abs(sig_fft)**2
    sample_freq = scipy.fftpack.fftfreq(sig.size, d=time_step)
    pos_mask = np.where(sample_freq > 0)
    freqs = sample_freq[pos_mask]
    peak_freq = freqs[power[pos_mask].argmax()]
    np.allclose(peak_freq, 1./period)
    
    high_freq_fft = sig_fft.copy()
    high_freq_fft[np.abs(sample_freq) > peak_freq] = 0
    filtered_sig = scipy.fftpack.ifft(high_freq_fft)
    return filtered_sig  #retourne un array qui représente le résultat du signal filtré sur un channel i 

In [ ]:
filt2=filtrage_file_channel_FFT(filt)

In [ ]:
plt.plot(raw_data[0][:90000])
plt.title("original signal")
plt.show()
plt.plot(filt[:90000])
plt.title("filtered signal : Lowpass Butterworth filter ")
plt.show()
plt.plot(filt2[:90000])
plt.title("filtered signal : FFT filter (smoothing filter) ")
plt.show()

In [ ]:
##########################################
#                                        #
#            DECOMPOS                    #
#                                        #
##########################################

In [ ]:

filt2

In [ ]:
import pandas as pd 
from pywt import wavedec
def decompose_signal_channel(filt2,i):
    #data = mne.io.read_raw_edf(file)
   # raw_data = data.get_data()
    channels = data.ch_names
    data1=filt2
    #channel_name=channels[channel]
    print(channels[i])
    # number of levels we are interested in
    level = 6

    # transpose the data because its a time-series package
    data_t = data1.transpose()

   # get the wavelet coefficients at each level in a list
    coeffs_list = wavedec(data1, wavelet='db4', level=level)
   
    coefficients=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1']
    
   
    L=[[]]
    for i in range(len(coefficients)):
      array=coeffs_list[i].flatten()
      list1=array.tolist()
      L.append(list1)
        
    L.remove(L[0])
    df = pd.DataFrame(columns=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1'])
    Series_coefficients=[]
    for  i in range(len(coeffs_list)):
     Series_coefficients.append(pd.Series(L[i]))
    
    
    for i in range(len(coefficients)):
      df[coefficients[i]]=Series_coefficients[i]
    
    return(df)

In [ ]:
df1=decompose_signal_channel(filt2,0)

In [ ]:
df1

In [ ]:
##########################################
#                                        #
#            FEATures eng                #
#                                        #
##########################################

In [ ]:
List_one=[1 for i in range(5182)]

In [ ]:
List_one

In [ ]:
Serie_one=pd.Series(List_one)

In [ ]:
Serie_one

In [ ]:
def minus_small(data):    
  # find the smallest value for each data column (channel)...
  min_val = data.min()
  # ...and subtract it from all the data in the column and add one
  List_one=[1 for i in range(5182)]
  Series_one=pd.Series(List_one)
  data_substract = np.subtract(data,min_val)
  data_modified=np.add(data_substract,List_one)


  return data

In [ ]:
minus_small(df1['A6'])

In [ ]:
df1.sum(axis=0)

In [ ]:
df_sum=df1.sum(axis=0)
df_sum

In [ ]:
absolute_sum=df1.sum(axis=0)

In [ ]:
absolute_sum

In [ ]:
def ave(data, output=False):
    # get the mean
    mean_data = data.mean()
    return(mean_data)

In [ ]:
ave(df1['A6'])

In [ ]:
def mean_abs(data, output=False):
    # get the mean of the absolute values
    mean_abs_data = data.abs().mean()
    return mean_abs_data

In [ ]:
mean_abs(df1['A6'])

In [ ]:
coefficients=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1']

In [ ]:
Lmean_abs =['mean_abs_A6', 'mean_abs_D6', 'mean_abs_D5', 'mean_abs_D4', 'mean_abs_D3', 'mean_abs_D2', 'mean_abs_D1']

In [ ]:
dfM_abs = pd.DataFrame(columns=Lmean_abs)

In [ ]:
def m_abs(df1) :  #retourne une liste des means abs a partir d une channel decompposee (dataframe )
    LmAbs=[]
    for i in range (0,7) :
       t=mean_abs(df1[coefficients[i]])
       LmAbs.append(t)
    return LmAbs

In [ ]:
LmAbs=mean_abs (df1)

In [ ]:
dfM_abs

In [ ]:
dfM_abs.loc[1,:]=LmAbs 

In [ ]:
dfM_abs

In [ ]:
raw_data.shape[0]

In [ ]:
for i in  range(raw_data.shape[0]):
  df1=df1=decompose_signal_channel(raw_data[i],i)
  LmAbs=m_abs(df1)
  dfM_abs.loc[i,:]=LmAbs 

  


In [ ]:
raw_data

In [ ]:
dfM_abs

In [ ]:
channels

In [ ]:
dfM_abs.insert(0, "channel", channels, True)

In [ ]:
dfM_abs

In [ ]:
dfM_abs

In [ ]:
#df_a=dfM_abs.loc[0]

In [ ]:
#df_a

In [ ]:
miindex = pd.MultiIndex.from_product([["x","y"], ["10","20"]],names=['row-foo', 'row-bar'])

In [ ]:
miindex

In [ ]:
micol = pd.MultiIndex.from_product([['a','b','c'], ["1","2"]],names=['col-foo', 'col-bar'])

In [ ]:
micol

In [ ]:
df5 = pd.DataFrame(index=miindex, columns=micol).sort_index().sort_index(axis=1)

In [ ]:
df5

In [ ]:
df5.loc[ ('x','10'),('a', '2')]

In [ ]:
miindex = pd.MultiIndex.from_product([["x","y"], ["10","20"]],names=['row-foo', 'row-bar'])